In [148]:
from __future__ import print_function
import cntk
import numpy as np
import scipy.sparse
import cntk.tests.test_utils
cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
cntk.cntk_py.set_fixed_random_seed(1) # fix the random seed so that examples produce the same results each time
from IPython.display import Image
import matplotlib.pyplot
%matplotlib inline
matplotlib.pyplot.rcParams['figure.figsize'] = (40,40)

# CNTK: A Guided Tour

Welcome to CNTK and the wonders of deep learning! This tutorial will give a brief overview of CNTK. It is meant for users that are new to CNTK but have some experience with deep neural networks.
The focus will be on how the basic steps of deep learning are done in CNTK.

To train a deep model, you will need to define your model structure, prepare your data so that it can be fed to CNTK, train the model and evaluate its accuracy, and deploy it.

This guided tour is organized as follows:

 * Defining your **model structure**
    * The CNTK programming model: Networks as Function Objects
    * CNTK's Data Model: Tensors and Sequences of Tensors
    * Your First CNTK Network: Logistic Regression
    * Your second CNTK Network: MNIST Digit Recognition
    * The Graph API: MNIST Digit Recognition Once More
 * Feeding your **data**
    * Small data sets that fit into memory: numpy/scipy arrays/
    * Large data sets: `MinibatchSource` class
    * Spoon-feeding data: your own minibatch loop
 * **Training**
    * Distributed Training
    * Logging
    * Checkpointing
    * Cross-validation based training control
    * Final evaluation
 * **Deploying** the model
    * From Python
    * From C++ and C#
    * From your own web service
    * Via an Azure web service
 * Conclusion
 

To run this tutorial, you will need CNTK 2.0 and ideally a CUDA-capable GPU (deep learning is no fun without GPUs).

# Defining Your Model Structure

So let us dive right in. Below we will introduce CNTK's programming model--*networks are function objects* and CNTK's data model. We will put that into action for logistic regression and MNIST digit recognition,
using CNTK's Functional API.
Lastly, we will replicate one example using CNTK's lower-level, TensorFlow/Theano-like graph API.

### The CNTK Programming Model: Networks are Function Objects

In CNTK, a neural network is a function object.
On one hand, a neural network in CNTK is just a function that you can call
to apply it to data.
On the other hand, a neural network contains learnable parameters
that can be accessed like object members.
Complicated function objects can be composed as hierarchies of simpler ones, which,
for example, represent layers.
The function-object approach is similar to Keras, Chainer, Dynet, Pytorch,
and the recent Sonnet.

The following illustrates the function-object approach with pseudo-code, using the example
of a fully-connected layer (called `Dense` in CNTK)::


In [149]:
# numpy *pseudo-code* for CNTK Dense layer (vastly simplified, e,g. no back-prop)
def Dense(out_dim, activation):
    # create the learnable parameters
    b = np.zeros(out_dim)
    W = np.ndarray((0,out_dim)) # input dimension is unknown
    # define the function itself
    def dense(x):
        if len(W) == 0:         # first call: reshape and initialize W
            W.resize((x.shape[-1], W.shape[-1]), refcheck=False)
            W[:] = np.random.randn(*W.shape) * 0.05
        return activation(x @ W + b)
    # return it as a function object: can be called & holds the parameters as members
    dense.W = W
    dense.b = b
    return dense

d = Dense(5, np.tanh)    # create the function object
y = d(np.array([1, 2]))  # apply it like a function
W = d.W                  # access member like an object
print('W =', d.W)
print('y =', y)

W = [[ 0.06484214  0.0668272  -0.06177861  0.05454117  0.00123526]
 [-0.08024099  0.04517105  0.06488221 -0.09801994 -0.02767334]]
y = [-0.0953493   0.15588782  0.06788126 -0.14056186 -0.05405867]


Note that the real CNTK function objects are not actual Python lambdas.
Rather, they are static graphs in C++,
wrapped in the Python class `Function` that exposes `__call__()` and `__getattr__()` methods.
However, CNTK allows to define networks as Python expressions and functions,
as long as they can be converted into static graphs of type `Function`.

The function object is CNTK's single abstraction used to represent different levels of neural networks, which
are only distinguished by convention:

 * **basic operations** without learnable parameters (e.g. `times()`, `__add__()`, `sigmoid()`...)
 * **layers** (`Dense()`, `Embedding()`, `Convolution()`...). Layers map one input to one output.
 * **recurrent step functions** (`LSTM()`, `GRU()`, `RNNStep()`). Step functions map a previous state and a new input to a new state.
 * **loss and metric** functions (`cross_entropy_with_softmax()`, `binary_cross_entropy()`, `squared_error()`, `classification_error()`...).
   In CNTK, losses and metric are not special, just functions.
 * **models**. Models are defined by the user and map features to predictions or scores, and is what gets deployed in the end.
 * **criterion function**. The criterion function maps (features, labels) to (loss, metric).
   The Trainer optimizes the loss by SGD, and logs the metric, which may be non-differentiable.

Higher-order layers compose objects into more complex ones, including:

 * **stacking** (`Sequential()`, `For()`)
 * **recurrence** (`Recurrence()`, `Fold()`, `Unfold()`...)

Users can use arbitrary Python expressions as CNTK function objects with `Function()`.
This is similar to Keras' `Lambda()`.
Expressions can be written as multi-line functions through decorator syntax (`@Function`).

Lastly, function objects enable parameter sharing. If you call the same
function object at multiple places, all invocations will naturally share the same learnable parameters.

In summary, the function object is CNTK's single abstraction for conveniently defining
simple and complex models, parameter sharing, and training objectives.

(Note that under the hood, CNTK uses a static graph,
and it is possible to define CNTK networks as graphs like TensorFlow and Theano, as discussed
further below.)

### CNTK's Data model: Sequences of Tensors

CNTK can operate on two types of data:

 * **tensors** (that is, N-dimensional arrays), dense or sparse
 * **sequences** of tensors

The distinction is that the shape of a tensor is static during operation,
while the length of a sequence depends on data.
Tensors have *static axes*, while a sequence has a *dynamic axis*.

In CNTK, categorical data is represented as sparse one-hot tensors, not as integer vectors.
This allows to write embeddings and loss functions in a unified fashion as matrix products.

CNTK adopts Python's type-annotation syntax to declare CNTK types.
For example,

 * `Tensor[(13,42)]` denotes a tensor with 13 rows and 42 columns, and
 * `Sequence[SparseTensor[300000]]` a sequence of sparse vectors, which for example could represent a word out of a 300k dictionary

Note that unlike Python type hints, this works with Python 2.7. There are two more data types: 'batch of tensors' and 'batch of sequences of tensors'.
These are used internally for minibatching, but we generally try to hide batching from the user:
We want users to think in tensors and sequences, and leave mini-batching to CNTK.
To this end, unlike other toolkits, CNTK transparently batches *sequences with different lengths*
into the same minibatch, and handles all the necessary padding and packing.
Workarounds like 'bucketing' are not needed.

### Your First CNTK Network: Simple Logistic Regression

Let us put all of this in action for a very simple example of logistic regression.
For this example, we create a synthetic data set of 2-dimensional normal-distributed 
data points, which should be classified into belonging to one of two classes.
Note that CNTK expects the labels as one-hot encoded.


In [150]:
input_dim_lr = 2    # classify 2-dimensional data
num_classes_lr = 2  # into one of two classes

# This example uses synthetic data from normal distributions, which we generate in the following.
#  X_lr[corpus_size,input_dim] - input data
#  Y_lr[corpus_size]           - labels (0 or 1), in one-hot representation
np.random.seed(0)
def generate_synthetic_data(N):
    Y = np.random.randint(size=N, low=0, high=num_classes_lr)  # labels
    X = (np.random.randn(N, input_dim_lr)+3) * (Y[:,None]+1)   # data
    # Our model expects float32 features, and cross-entropy expects one-hot encoded labels.
    Y = scipy.sparse.csr_matrix((np.ones(N,np.float32), (range(N), Y)), shape=(N, num_classes_lr))
    X = X.astype(np.float32)
    return X, Y
X_train_lr, Y_train_lr = generate_synthetic_data(20000)
X_test_lr,  Y_test_lr  = generate_synthetic_data(1024)
X_train_lr, Y_train_lr.todense() # let's have a peek
print(X_train_lr[:4])
print(Y_train_lr[:4].todense())

[[ 2.2741797   3.56347561]
 [ 5.12873602  5.79089499]
 [ 1.3574543   5.5718112 ]
 [ 3.54340553  2.46254587]]
[[ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


We now define the model function. The model function maps input data to predictions.
It is the final product of the training process.
In this example, we use the simplest of all models: logistic regression.

In [151]:
model_lr = cntk.layers.Dense(num_classes_lr, activation=None)

Next, we define the criterion function. The criterion function is
the harness via which the trainer uses to optimize the model:
It maps (input vectors, labels) to (loss, metric).
The loss is used for the SGD updates. We choose cross entropy.
Specifically, `cross_entropy_with_softmax()` first applies
the `softmax()` function to the network's output, as
cross entropy expects probabilities.
We do not include `softmax()` in the model function itself, because
it is not necessary for using the model.
As the metric, we count classification errors (this metric is not differentiable).

We define criterion function as Python code and convert it to a `Function` object.
A single expression can be written as `Function(lambda x, y: `*expression of x and y*`)`,
similar to Keras' `Lambda()`.
To avoid evaluating the model twice, we use a Python function definition
with decorator syntax. Lastly, this is a good time to tell CNTK about the
data types of our inputs, which is done via the decorator `@Function.with_signature(`*argument types*`)`:

In [152]:
@cntk.Function.with_signature(cntk.layers.Tensor[input_dim_lr], cntk.layers.SparseTensor[num_classes_lr])
def criterion_lr(data, label_one_hot):
    z = model_lr(data)  # apply model. Computes a non-normalized log probability for every output class.
    loss   = cntk.cross_entropy_with_softmax(z, label_one_hot) # this applies softmax to z under the hood
    metric = cntk.classification_error(z, label_one_hot)
    return loss, metric
print(criterion_lr)
print(model_lr.W.value) # W now has known shape and thus gets initialized

Composite(data: Tensor[2], label_one_hot: SparseTensor[2]) -> Tuple[Tensor[1], Tensor[1]]
[[-0.89681542 -0.89061725]
 [-0.11949861 -1.17324626]]


The decorator will 'compile' the function into a static graph
by calling `criterion()` passing it input variables of the given data types as arguments.
Thus, `criterion` is not a Python function but a static graph in a CNTK `Function` object.

We are now ready to train our model.

In [153]:
learner = cntk.sgd(model_lr.parameters, cntk.learning_rate_schedule(0.1, cntk.UnitType.minibatch))
progress_writer = cntk.logging.ProgressPrinter(50)

criterion_lr.train((X_train_lr, Y_train_lr), parameter_learners=[learner], callbacks=[progress_writer])

print(model_lr.W.value) # peek at updated W

Learning rate per minibatch: 0.1
 Minibatch[   1-  50]: loss = 0.663274 * 1600, metric = 37.31% * 1600;
 Minibatch[  51- 100]: loss = 0.481867 * 1600, metric = 20.56% * 1600;
 Minibatch[ 101- 150]: loss = 0.402196 * 1600, metric = 12.94% * 1600;
 Minibatch[ 151- 200]: loss = 0.386619 * 1600, metric = 13.75% * 1600;
 Minibatch[ 201- 250]: loss = 0.328646 * 1600, metric = 9.19% * 1600;
 Minibatch[ 251- 300]: loss = 0.301831 * 1600, metric = 9.50% * 1600;
 Minibatch[ 301- 350]: loss = 0.299345 * 1600, metric = 9.44% * 1600;
 Minibatch[ 351- 400]: loss = 0.279577 * 1600, metric = 8.94% * 1600;
 Minibatch[ 401- 450]: loss = 0.281061 * 1600, metric = 8.25% * 1600;


KeyboardInterrupt: 

RuntimeError: SWIG director method error.

The `learner` is the object that actually performs the model update. Alternative learners include `momentum_sgd()` and `adam()`. The `progress_writer` is a stock logging callback that prints the output you see above, and can be replaced by your own
or the stock `TensorBoardProgressWriter`to visualize training progress using TensorBoard.

The `train()` function is feeding our data `(X_train_lr, Y_train_lr)` minibatch by minibatch to the model and updates it, where the data is a tuple in the same order as the arguments of `criterion_mn()`.

Let us test how we are doing on our test set (this will also run minibatch by minibatch).

In [ ]:
test_metric = criterion_lr.test((X_test_lr, Y_test_lr), callbacks=[progress_writer]).metric

And lastly, let us run a few samples through our model and see how it is doing.
Oops, `criterion` knew the input types, but `model_lr` does not,
so we tell it using `update_signature()`.

In [ ]:
model_lr.update_signature(cntk.layers.Tensor[input_dim_lr])
print(model_lr)

Now we can call it like any Python function:

In [ ]:
z = model_lr(X_test_lr[:25])
print("Label    :", [label.todense().argmax() for label in Y_test_lr[:25]])
print("Predicted:", [z[i,:].argmax() for i in range(len(z))])

### Your Second CNTK Network: MNIST Digit Recognition

Let us do the same thing as above on an actual task--the MNIST benchmark, which is sort of the "hello world" of deep learning.
The MNIST task is to recognize scans of hand-written digits. We first download and prepare the data.

In [ ]:
input_shape_mn = (28, 28)  # MNIST digits are 28 x 28
num_classes_mn = 10        # classify as one of 10 digits

# Fetch the MNIST data. Best done with scikit-learn.
try:
    from sklearn import datasets, utils
    mnist = datasets.fetch_mldata("MNIST original")
    X, Y = mnist.data / 255.0, mnist.target
    X_train_mn, X_test_mn = X[:60000].reshape((-1,28,28)), X[60000:].reshape((-1,28,28))
    Y_train_mn, Y_test_mn = Y[:60000].astype(int), Y[60000:].astype(int)
except: # workaround if scikit-learn is not present
    import requests, io, gzip
    X_train_mn, X_test_mn = (np.fromstring(gzip.GzipFile(fileobj=io.BytesIO(requests.get('http://yann.lecun.com/exdb/mnist/' + name + '-images-idx3-ubyte.gz').content)).read()[16:], dtype=np.uint8).reshape((-1,28,28)).astype(np.float32) / 255.0 for name in ('train', 't10k'))
    Y_train_mn, Y_test_mn = (np.fromstring(gzip.GzipFile(fileobj=io.BytesIO(requests.get('http://yann.lecun.com/exdb/mnist/' + name + '-labels-idx1-ubyte.gz').content)).read()[8:], dtype=np.uint8).astype(int) for name in ('train', 't10k'))

# Shuffle the training data.
np.random.seed(0) # always use the same reordering, for reproducability
idx = np.random.permutation(len(X_train_mn))
X_train_mn, Y_train_mn = X_train_mn[idx], Y_train_mn[idx]

# Further split off a cross-validation set
X_train_mn, X_cv_mn = X_train_mn[:54000], X_train_mn[54000:]
Y_train_mn, Y_cv_mn = Y_train_mn[:54000], Y_train_mn[54000:]

# Our model expects float32 features, and cross-entropy expects one-hot encoded labels.
Y_train_mn, Y_cv_mn, Y_test_mn = (scipy.sparse.csr_matrix((np.ones(len(Y),np.float32), (range(len(Y)), Y)), shape=(len(Y), 10)) for Y in (Y_train_mn, Y_cv_mn, Y_test_mn))
X_train_mn, X_cv_mn, X_test_mn = (X.astype(np.float32) for X in (X_train_mn, X_cv_mn, X_test_mn))

# have a peek
matplotlib.pyplot.rcParams['figure.figsize'] = (0.5,0.5)
matplotlib.pyplot.axis('off')
_ = matplotlib.pyplot.imshow(X_train_mn[13], cmap="gray_r")

Let's define the CNTK model function to map (28x28)-dimensional images to a 10-dimensional score vector. We wrap that in a function so that later in this tutorial we can easily recreate it.

In [ ]:
def create_model_mn():
    with cntk.layers.default_options(activation=cntk.ops.relu, pad=False):
        return cntk.layers.Sequential([
            cntk.layers.Convolution2D((5,5), num_filters=32, reduction_rank=0, pad=True), # reduction_rank=0 for B&W images
            cntk.layers.MaxPooling((2,2), strides=(2,2)),
            cntk.layers.Convolution2D((3,3), num_filters=48),
            cntk.layers.MaxPooling((2,2), strides=(2,2)),
            cntk.layers.Convolution2D((3,3), num_filters=64),
            cntk.layers.Dense(96),
            cntk.layers.Dropout(dropout_rate=0.5),
            cntk.layers.Dense(num_classes_mn, activation=None) # no activation in final layer (softmax is done in criterion)
        ])
model_mn = create_model_mn()

This model is a tad bit more complicated! It consists of several convolution-pooling layeres and two
fully-connected layers for classification which is typical for MNIST. This demonstrates several aspects of CNTK's Functional API.

First, we create each layer using a function from CNTK's layers library (`cntk.layers`).

Second, the higher-order layer `Sequential()` creates a new function that applies all those layers
one after another. This is known [forward function composition](https://en.wikipedia.org/wiki/Function_composition).
Note that unlike some other toolkits, you cannot `Add()` more layers afterwards to a sequential layer.
CNTK's `Function` objects are immutable, besides their learnable parameters (to edit a `Function` object, you can `clone()` it).
If you prefer that style, create your layers as a Python list and pass that to `Sequential()`.

Third, the context manager `default_options()` allows to specify defaults for various optional arguments to layers,
such as that the activation function is always `relu`, unless overriden.

Lastly, note that `relu` is passed as the actual function, not a string.
Any function can be an activation function.
It is also allowed to pass a Python lambda directly, for example relu could also be
realized manually by saying `activation=lambda x: cntk.ops.element_max(x, 0)`.

The criterion function is defined like in the previous example, to map maps (28x28)-dimensional features and according
labels to loss and metric.

In [ ]:
@cntk.Function.with_signature(cntk.layers.Tensor[input_shape_mn], cntk.layers.SparseTensor[num_classes_mn])
def criterion_mn(data, label_one_hot):
    z = model_mn(data)
    loss   = cntk.cross_entropy_with_softmax(z, label_one_hot)
    metric = cntk.classification_error(z, label_one_hot)
    return loss, metric

For the training, let us throw momentum into the mix.

In [ ]:
N = len(X_train_mn)
lrs = cntk.learning_rate_schedule([0.001]*12 + [0.0005]*6 + [0.00025]*6 + [0.000125]*3 + [0.0000625]*3 + [0.00003125], cntk.learners.UnitType.sample, epoch_size=N)
momentums = cntk.learners.momentum_as_time_constant_schedule([0]*5 + [1024], epoch_size=N)
minibatch_sizes = cntk.minibatch_size_schedule([256]*6 + [512]*9 + [1024]*7 + [2048]*8 + [4096], epoch_size=N)

learner = cntk.learners.momentum_sgd(model_mn.parameters, lrs, momentums)

This looks unusual in several aspects.
First, the learning rate is specified as a list (`[0.001]*12 + [0.0005]*6 +`...). Together with the `epoch_size` parameter, this tells CNTK to use 0.001 for 12 epochs, and then continue with 0.005 for another 6, etc.

Second, the learning rate has `cntk.learners.UnitType.sample` (vs. `cntk.UnitType.minibatch`). This tells CNTK to apply the learning rate not to the minibatch-*average*, but to each sample (or, the minibatch-*sum*). To get the same convergence, the learning rate must be divided by the minibatch size, hence the small value.

Third, momentum is specified as a time constant of 1024. This says that for a minibatch size of 64, after 1024/64=16 minibatches, each sample gradient still contributes with a weight of 1/e.

But why?? In our experience, the effect of learning rates and momentum on training convergence, when specified this way, tends to be independent of the minibatch size within a wide range. The minibatch size now just means how 'outdated' the model is before we update it.
The minibatch size is crucial for efficiency of GPUs and parallel training, and this way it can be tuned while keeping learning rate and momentum the same. We find that it is possible to crank up the minibatch size to much larger values during the training. The minibatch-size schedule yields 6-times speed-up (!) towards the end (on a Titan-X).

Having this out of the way, let us now train the model. On a Titan-X, this will run for about a minute.

In [ ]:
progress_writer = cntk.logging.ProgressPrinter()
criterion_mn.train((X_train_mn, Y_train_mn), minibatch_size=minibatch_sizes,
                   max_epochs=40, parameter_learners=[learner], callbacks=[progress_writer])
test_metric = criterion_mn.test((X_test_mn, Y_test_mn), callbacks=[progress_writer]).metric

### Graph API Example: MNIST Digit Recognition Again

CNTK also allows networks to be written in graph style like TensorFlow and Theano. The following defines the same model and criterion function as above, and you would get the same result if you had run the training above with the definition below (if you try that now, results will differ due to random initialization).

In [ ]:
images = cntk.input_variable(input_shape_mn, name='images')
with cntk.layers.default_options(activation=cntk.ops.relu, pad=False):
    r = cntk.layers.Convolution2D((5,5), num_filters=32, reduction_rank=0, pad=True)(images)
    r = cntk.layers.MaxPooling((2,2), strides=(2,2))(r)
    r = cntk.layers.Convolution2D((3,3), num_filters=48)(r)
    r = cntk.layers.MaxPooling((2,2), strides=(2,2))(r)
    r = cntk.layers.Convolution2D((3,3), num_filters=64)(r)
    r = cntk.layers.Dense(96)(r)
    r = cntk.layers.Dropout(dropout_rate=0.5)(r)
    model_mn = cntk.layers.Dense(num_classes_mn, activation=None)(r)

label_one_hot = cntk.input_variable(num_classes_mn, is_sparse=True, name='labels')
loss   = cntk.cross_entropy_with_softmax(model_mn, label_one_hot)
metric = cntk.classification_error(model_mn, label_one_hot)
criterion_mn = cntk.combine([loss, metric])
print(criterion_mn)

# Feeding Your Data

Once you have decided your model structure and defined it, you are facing the question on feeding
your training data to the CNTK training process.

The above examples simply feed the data as numpy/scipy arrays.
That is only one of three ways CNTK provides for feeding data to the trainer:

 1. As **numpy/scipy arrays**, for small data sets that can just be loaded into RAM.
 2. Through instances of **CNTK's MinibatchSource class**, for large data sets that do not fit into RAM.
 3. Through an **explicit minibatch-loop** when the above do not apply.

### 1. Feeding Data Via Numpy/Scipy Arrays

The `train()` and `test()` functions accept a tuple of numpy or scipy arrays for their `minibatch_source` arguments.
The tuple members must be in the same order as the arguments of the `criterion` function that `train()` or `test()` are called on.
For dense tensors, use numpy arrays, while sparse data should have the type `scipy.sparse.csr_matrix`.

Each of the arguments should be a Python list of numpy/scipy arrays, where each list entry represents a data item. For arguments declared as `Sequence[...]`, the first axis of the numpy/scipy array is the sequence length, while the remaining axes are the shape of each token of the sequence. Arguments that are not sequences consist of a single tensor. The shapes, data types (`np.float32/float64`) and sparseness must match the argument types as declared in the criterion function.

As an optimization, arguments that are not sequences can also be passed as a single large numpy/scipy array (instead of a list). This is what is done in the examples above.

Note that it is the responsibility of the user to randomize the data.

### 2. Feeding Data Using the `MinibatchSource` class for Reading Data

Production-scale training data sometimes does not fit into RAM. For example, a typical speech corpus may be several hundred GB large. For this case, CNTK provides the `MinibatchSource` class, which provides:

 * A **chunked randomization algorithm** that holds only part of the data in RAM at any given time.
 * **Distributed reading** where each worker reads a different subset.
 * A **transformation pipeline** for images and image augmentation.
 * **Composability** across multiple data types (e.g. image captioning).

At present, the `MinibatchSource` class implements a limited set of data types in the form of "deserializers":

 * **Images** (`ImageDeserializer`).
 * **Speech files** (`HTKFeatureDeserializer`, `HTKMLFDeserializer`).
 * Images in CNTK's **canonical text format (CTF)**, which encodes any of CNTK's data types in a human-readable text format.

The following example shows how to instantiate an image reader that crops images to 80% at random offsets, and scales the result to 32x32 pixels.

In [ ]:
image_width, image_height, num_channels = (32, 32, 3)
num_classes = 1000
def create_image_reader(map_file, mean_file, train):
    transforms = []
    # train uses data augmentation (translation only)
    if train:
        transforms += [
            cntk.io.transforms.crop(crop_type='randomside', side_ratio=0.8) 
        ]
    transforms += [
        cntk.io.transforms.scale(width=image_width, height=image_height, channels=num_channels, interpolations='linear'),
        cntk.io.transforms.mean(mean_file)
    ]
    # deserializer
    return cntk.io.MinibatchSource(cntk.io.ImageDeserializer(map_file, cntk.io.StreamDefs(
        features = cntk.io.StreamDef(field='image', transforms=transforms),
        labels   = cntk.io.StreamDef(field='label', shape=num_classes)
    )))

### 3.  Feeding Data Via an Explicit Minibatch Loop

Instead of feeding your data as a whole to CNTK's `train()` and `test()` functions which implement a minibatch loop internally,
you can realize your own minibatch loop and call the lower-level APIs `train_minibatch()` and `test_minibatch()`.
This is useful when your data is not in a form suitable for the above, such as being generated on the fly as in variants of reinforcement learning. The `train_minibatch()` and `test_minibatch()` methods require you to instantiate an object of class `Trainer` that takes a subset of the arguments of `train()`. The following implements the logistic-regression example from above through explicit minibatch loops:

In [ ]:
# Recreate the model, so that we can start afresh. This is a direct copy from above.
model_lr = cntk.layers.Dense(num_classes_lr, activation=None)
@cntk.Function.with_signature(cntk.layers.Tensor[input_dim_lr], cntk.layers.SparseTensor[num_classes_lr])
def criterion_lr(data, label_one_hot):
    z = model_lr(data)  # apply model. Computes a non-normalized log probability for every output class.
    loss   = cntk.cross_entropy_with_softmax(z, label_one_hot) # this applies softmax to z under the hood
    metric = cntk.classification_error(z, label_one_hot)
    return loss, metric

# Create the learner; same as above.
learner = cntk.sgd(model_lr.parameters, cntk.learning_rate_schedule(0.1, cntk.UnitType.minibatch))

# This time we must create a Trainer instance ourselves.
trainer = cntk.Trainer(None, criterion_lr, [learner], [cntk.logging.ProgressPrinter(50)])

# Train the model by spoon-feeding minibatch by minibatch.
minibatch_size = 32
for i in range(0, len(X_train_lr), minibatch_size): # loop over minibatches
    x = X_train_lr[i:i+minibatch_size] # get one minibatch worth of data
    y = Y_train_lr[i:i+minibatch_size]
    trainer.train_minibatch({criterion_lr.arguments[0]: x, criterion_lr.arguments[1]: y})  # update model from one minibatch
trainer.summarize_training_progress()

# Test error rate minibatch by minibatch
evaluator = cntk.Evaluator(criterion_lr.outputs[1], [progress_writer]) # metric is the second output of criterion_lr()
for i in range(0, len(X_test_lr), minibatch_size): # loop over minibatches
    x = X_test_lr[i:i+minibatch_size] # get one minibatch worth of data
    y = Y_test_lr[i:i+minibatch_size]
    evaluator.test_minibatch({criterion_lr.arguments[0]: x, criterion_lr.arguments[1]: y})  # test one minibatch
evaluator.summarize_test_progress()

# Training and Evaluating

We have already seen several examples of training and testing. In this section, we want to walk you through advanced options of the `train()` function:

 1. **Distributed Training** on mutliple GPUs using MPI.
 2. **Progress Tracking** callback, for output to log file and TensorBoard visualization.
 3. **Checkpointing** callback to checkpoint intermediate models.
 4. **Cross-validation** callback for CV-based training control for learning-rate adjustment and early stopping.
 5. **Testing** callback to test the final model.

All of these are provided through the `callbacks` parameter to `train()`. Callbacks are actions that the `train()` function
executes periodically, typically every epoch.
The `callbacks` parameter is a list of objects, where the object type decides the specific callback action.

### 1. Distributed Training

CNTK makes distributed training easy. Out of the box, it supports three methods of distributed training:

 * Simple **data-parallel** training.
 * **1-bit SGD**.
 * **BlockMomentum**.

Simple **data-parallel** training distributes each minibatch over N worker processes, where each process utilizes one GPU.
After each minibatch, sub-minibatch gradients from all workers are aggregated before updating each model copy.
This is often sufficient for convolutional networks, which have a high computation/communication ratio.

**1-bit SGD** uses 1-bit data compression with residual feedback to speed up data-parallel training
by reducing the data exchanges to 1 bit per gradient value.
To avoid affecting convergence, each worker keeps a quantization-error residual which is added to the next minibatch's
gradient. This way, all gradient values are eventually transmitted with full accuracy, albeit at a delay.
This method has been found effective for networks where communication cost becomes the dominating factor,
such as full-connected networks and some recurrent ones.
This method has been found to only minimally degrade accuracy at good speed-ups.

**BlockMomentum** improves communication bandwidth by exchanging gradients only every N minibatches.
To avoid affecting convergence, BlockMomentum combines "model averaging" with the residual technique of 1-bit SGD:
After N minibatches, block gradients are aggregated across workers, and added to all model copies at weight of 1/N,
while a residual keeps (N-1)/N times the block gradient, which is added to the next block gradient, which
then is in turn applied at a weight of 1/N and so on.

Processes are started with and communicate through MPI. Hence, CNTK's distributed training
works both within a single server and across multiple servers.
All you need to do is

 * wrap your learner inside a `distributed_learner` object
 * execute the Python script using `mpiexec`

The following creates a distributed learner for 1-bit SGD where distributed training kicks in after a warm-start period of one epoch:

In [ ]:
learner = cntk.train.distributed.data_parallel_distributed_learner(learner, distributed_after=1, num_quantization_bits=1)

Now execute this script with `mpiexec` (the distributed learner has no effect when running without MPI). The command is somewhat system-dependent. This is the command for Microsoft MPI for Windows.

`mpiexec -n 4 -lines python -u `* your_python_script*`.py`

### 2. Progress Tracking

Progress is tracked through progress trackers, and allows to log to a logfile as well as a TensorBoard event log. Users can also implement their own progress tracker classes.
Progress tracking gets invoked in two ways:

 * After every N minibatches, the average loss and metric over these minibatches is printed.
   Choose N>1 to avoid unnecessary GPU synchronizations.
 * After completing an epoch (given by epoch length), a summary of the epoch is printed,
   including loss and metric averaged over the entire epoch, as well as a speed measurement.

The number of minibatches N is a parameter to the `ProgressPrinter` constructor.
Since this information is more relevant at the start of the training, it is possible print every single
minibatch loss/metric for the first few minibatches.
Optionally, if a pathname is given the log will be redirected there.
The following shows how to print statistics for the first 10 minibatches, then for every 50th, to a log file.

In [ ]:
progress_writer = cntk.logging.ProgressPrinter(50, first=10, log_to_file='my.log') # show progress every 50 minibatches

Another progress tracker is the `TensorBoardProgressWriter`. The following example saves log information every 50 minibatches, as well as the model itself (in this case our MNIST model), to a given directory in TensorBoard format:

In [ ]:
tensorboard_writer = cntk.logging.TensorBoardProgressWriter(50, log_dir='my_tensorboard_logdir', model=model_mn)

### 3 Checkpointing

The following creates a callback that writes a checkpoint file every epoch. By passing `restore=True`, training will automatically be restarted at the latest available checkpoint:

In [ ]:
epoch_size=len(X_train_mn)
checkpoint_callback_config = cntk.CheckpointConfig('model_mn.cmf', epoch_size, restore=False)

### 4. Cross-validation based Training Control

The cross-validation callback tells CNTK to periodically evaluate the model on a cross-validation data set,
and then call a user-specified callback function, which can then update the learning rate of return `False` to indicate early stopping. The following example halves the learning rate each time the cross-validation metric improved less than 5% relative, and stops at the 7th adjustments.

In [ ]:
prev_metric = 1 # metric from previous call to the callback. At very beginning, error rate is 100%.
def adjust_lr_callback(index, average_error, cv_num_samples, cv_num_minibatches):
    global prev_metric
    if (prev_metric - average_error) / prev_metric < 0.05: # relative gain must reduce metric by at least 5% rel
        learner.reset_learning_rate(cntk.learning_rate_schedule(learner.learning_rate() / 2, cntk.learners.UnitType.sample))
        if learner.learning_rate() < lrs[0] / (2**7-0.1): # we are done after the 6-th LR cut
            print("Learning rate {} too small. Training complete.".format(learner.learning_rate()))
            return False # means we are done
        print("Improvement of metric from {:.3f} to {:.3f} insufficient. Halving learning rate to {}.".format(prev_metric, average_error, learner.learning_rate()))
    prev_metric = average_error
    return True # means continue

cv_callback_config = cntk.CrossValidationConfig((X_cv_mn, Y_cv_mn), 3*epoch_size, minibatch_size=256,
                                                callback=adjust_lr_callback, criterion=criterion_mn)

### 5. Test at End of Epoch

This callback instructs CNTK to evaluate the model at the end on a given test set. Example:

In [ ]:
test_callback_config = cntk.TestConfig((X_test_mn, Y_test_mn), criterion=criterion_mn)

### Putting it all Together

Let us now put all of the above examples together into a single training. This runs our MNIST example from above with logging, TensorBoard events, checkpointing, CV-based training control, and a final test.

Unfortunately, we cannot use MPI from a Jupyter notebook; hence, the the `distributed_learner` has no effect.
You can find the same example
as a standalone Python script under `Examples/1stSteps/MNIST_Complex_Training.py` to run under MPI, for example under MSMPI as

`mpiexec -n 4 -lines python -u Examples/1stSteps/MNIST_Complex_Training.py`

In [ ]:
# Recreate our model.
model_mn = create_model_mn()
@cntk.Function.with_signature(cntk.layers.Tensor[input_shape_mn], cntk.layers.SparseTensor[num_classes_mn])
def criterion_mn(data, label_one_hot):
    z = model_mn(data)
    loss   = cntk.cross_entropy_with_softmax(z, label_one_hot)
    metric = cntk.classification_error(z, label_one_hot)
    return loss, metric

# Create distributed learner and callbacks.
learner = cntk.learners.momentum_sgd(model_mn.parameters, lrs, momentums)
learner = cntk.train.distributed.data_parallel_distributed_learner(learner, distributed_after=1, num_quantization_bits=1)
progress_writer = cntk.logging.ProgressPrinter(50, first=10, log_to_file='my.log') # show progress every 50 minibatches
tensorboard_writer = cntk.logging.TensorBoardProgressWriter(50, log_dir='my_tensorboard_logdir',
                                                            rank=cntk.train.distributed.Communicator.rank(), model=criterion_mn)
checkpoint_callback_config = cntk.CheckpointConfig('model_mn.cmf', len(X_train_mn), preserve_all=True, restore=False)
cv_callback_config = cntk.CrossValidationConfig((X_cv_mn, Y_cv_mn), 3*epoch_size, minibatch_size=256,
                                                callback=adjust_lr_callback, criterion=criterion_mn)
test_callback_config = cntk.TestConfig((X_test_mn, Y_test_mn), criterion=criterion_mn)

# train it
callbacks = [progress_writer, tensorboard_writer, checkpoint_callback_config, cv_callback_config, test_callback_config]
#callbacks = [progress_writer, checkpoint_callback_config, cv_callback_config, test_callback_config]
progress = criterion_mn.train((X_train_mn, Y_train_mn), minibatch_size=minibatch_sizes,
                              max_epochs=50, parameter_learners=[learner], callbacks=callbacks)

# Deploying your Model

Your ultimate purpose of training a deep neural network is to deploy it as part of your own program or product.
Since this involves programming languages other than Python,
we will only give a high-level overview here, and refer you to specific examples.

Once you completed training your model, it can be deployed in a number of ways.

 * Directly in **Python**.
 * From any other language that CNTK supports, including **C++** and **C#**.
 * From **your own web serive**.
 * Through a web service deployed to **Microsoft Azure**.

The first step in all cases is to make sure your model's input types are known, and then to save your model to disk after training:

In [ ]:
model_mn.update_signature(cntk.layers.Tensor[input_shape_mn])
model_mn.save('mnist.cmf')

Deploying your model in a Python-based program is easy: Since networks are function objects that are callable, like a function, simply load the model, and call it with inputs, as we have already shown above:

In [ ]:
# At program start, load the model.
classify_digit = cntk.Function.load('mnist.cmf')

# To apply model, just call it.
image_input = X_test_mn[8345]        # (pick a random test digit for illustration)
scores = classify_digit(image_input) # call the model function with the input data
image_class = scores.argmax()        # find the highest-scoring class

# And that's it. Let's have a peek at the result
print('Recognized as:', image_class)
matplotlib.pyplot.axis('off')
_ = matplotlib.pyplot.imshow(image_input, cmap="gray_r")

Models can be deployed directly from programs written in other programming languages for which bindings exist.
Please see the following example programs for an example similar to the Python one above:

 * C++: `Examples/Evaluation/CNTKLibraryCPPEvalCPUOnlyExamples/CNTKLibraryCPPEvalCPUOnlyExamples.cpp`
 * C#: `Examples\Evaluation\CNTKLibraryCSEvalCPUOnlyExamples\CNTKLibraryCSEvalExamples.cs`

To deploy a model from your own web service, load and invoke the model in the same way.

To deploy a model via an Azure web service, follow this tutorial: `Examples\Evaluation\CNTKAzureTutorial01`

# Conclusion

This tutorial provided an overview of the five main tasks of creating and using a deep neural network with CNTK.

We first examined CNTK's Functional programming and its tensor/sequence-based data model.
Then we considered the possible ways of feeding data to CNTK, including directly from RAM,
through CNTK's data-reading infrastructure (`MinibatchSource`), and spoon-feeding through a custom minibatch loop.
We then took a look at CNTK's advanced training options, including distributed training, logging to TensorBoard, checkpointing, CV-based training control, and final model evaluation.
Lastly, we briefly looked into model deployment.

We hope this guided your have you a good starting point for your own ventures with CNTK. Please enjoy!